# Open AI Dall-E 2 기반 이미지 생성

In [ ]:
!pip install openai

In [ ]:
from google.colab import userdata
import openai
import os


# OpenAI API 키 설정 (실제 사용할 때는 노출되지 않도록 환경 변수 활용)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# API 키 로드
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
import IPython.display as display
import requests
from PIL import Image
from io import BytesIO


# 최신 DALL·E API를 사용하여 이미지 생성
response = openai.images.generate(
    model="dall-e-2",  # DALL·E 2 모델 사용
    prompt="Draw a night view of the city",  # 원하는 이미지 설명
    n=1,  # 생성할 이미지 개수 (1~4 가능)
    size="1024x1024"  # 이미지 크기 ("1024x1024", "1792x1024", "1024x1792" 가능)
)

# 생성된 이미지 URL 출력
image_url = response.data[0].url
print("Generated Image URL:", image_url)

# 이미지 다운로드 및 표시
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
display.display(img)



# Stable Diffusion 기반 이미지 생성

In [ ]:
!pip install diffusers transformers accelerate torch safetensors


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# 모델 ID 설정 (Stable Diffusion v1.5 사용)
model_id = "CompVis/stable-diffusion-v1-4"

# 모델 로드 (GPU 사용 가능하면 GPU로 설정)
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")  # GPU 사용 (Colab에서 반드시 필요)

print("✅ 모델 로드 완료")


In [ ]:
import IPython.display as display
from PIL import Image

# 생성할 이미지 설명 (Prompt)
prompt = "Draw a night view of the city"

# 이미지 생성
image = pipe(prompt).images[0]

# 이미지 저장
image.save("generated_image.png")
print("✅ 이미지 저장 완료")

# 이미지 출력
img = Image.open("generated_image.png")
display.display(img)
print("✅ 이미지 출력 완료")

# 이미지 생성형 서비스 심화

In [ ]:
!pip install diffusers transformers accelerate torch safetensors

In [ ]:
# GPU가 활성화되어 있는지 확인
import torch

if torch.cuda.is_available():
    print("GPU is 사용 가능!")
    print(f"사용 중인 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU를 사용할 수 없습니다. 런타임 유형을 GPU로 변경하세요!")


In [ ]:
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"

# 모델 다운로드 및 로드
pipeline = StableDiffusionPipeline.from_pretrained(model_id)
pipeline.to("cuda")  # GPU 사용


In [ ]:
from IPython.display import display

prompt = "A futuristic cityscape with neon lights"
image = pipeline(prompt).images[0]

display(image)  # 코랩에서 직접 출력 가능


In [ ]:
from IPython.display import display

prompt = "A concept car designed for the year 2050, ultra-detailed, photorealistic"
images = pipeline(prompt, num_images_per_prompt=4).images

# 생성된 이미지 출력
for idx, img in enumerate(images):
    display(img)
    img.save(f"generated_image_{idx}.png")


In [ ]:
from IPython.display import display

prompts = [
    "A luxury sports car with a futuristic design, photorealistic",
    "A classic muscle car in a sci-fi setting, cinematic lighting",
    "A concept car designed by a top automobile designer, ultra-detailed"
]

images = [pipeline(prompt).images[0] for prompt in prompts]

# 생성된 이미지 출력
for idx, img in enumerate(images):
    img.show()
    img.save(f"generated_image_{idx}.png")


In [ ]:
from IPython.display import display

prompt = "A high-tech self-driving electric vehicle in a futuristic city, 8K ultra HD"

image = pipeline(prompt, guidance_scale=7.5, num_inference_steps=50).images[0]
display(image)


In [ ]:
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"

# ✅ Half Precision (float16)으로 로드하여 메모리 사용량 감소
pipeline = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # 반정밀도 사용
    low_cpu_mem_usage=True  # CPU 메모리 사용량도 최적화
)

pipeline.to("cuda")  # GPU 사용

In [ ]:
pipeline.save_pretrained("./my_pipeline")
print("✅ 모델 저장 완료")

In [ ]:
from huggingface_hub import login

# Hugging Face 로그인
login(token=userdata.get("HF_TOKEN"))

# 모델 업로드
# pipeline.push_to_hub("your_huggingface_username/my-stable-diffusion")
pipeline.push_to_hub("allaboutswing/my-stable-diffusion")
print("✅ 모델 업로드 완료!")


# FastAPI 기반 Stable Diffusion REST API 구축



In [ ]:
!pip install fastapi uvicorn pyngrok diffusers transformers torch torchvision pillow


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# ✅ Stable Diffusion 모델 다운로드 및 로드
model_id = "CompVis/stable-diffusion-v1-4"

pipeline = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16  # ✅ Half Precision으로 메모리 절약
).to("cuda")  # ✅ GPU 사용


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from io import BytesIO
from PIL import Image
import base64

app = FastAPI()

# 요청 데이터 모델 정의
class RequestData(BaseModel):
    prompt: str
    num_inference_steps: int = 50
    guidance_scale: float = 7.5

def image_to_base64(image: Image) -> str:
    """이미지를 Base64 문자열로 변환"""
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

@app.post("/generate/")
async def generate_image(data: RequestData):
    """
    Stable Diffusion을 사용하여 이미지 생성 API
    """
    with torch.cuda.amp.autocast():  # ✅ Mixed Precision 최적화
        image = pipeline(
            data.prompt,
            num_inference_steps=data.num_inference_steps,
            guidance_scale=data.guidance_scale
        ).images[0]

    # 이미지를 Base64 문자열로 변환하여 반환
    base64_image = image_to_base64(image)
    return {"image": base64_image}


In [ ]:
import os
from google.colab import userdata

# 환경 변수로 저장
os.environ["NGROK_TOKEN"] = userdata.get("NGROK_TOKEN")

!pip install pyngrok

# https://dashboard.ngrok.com/get-started/your-authtoken authtoken 획득
!ngrok config add-authtoken $NGROK_TOKEN


# !ngrok config add-authtoken {AUTH_TOKEN}


In [ ]:
from pyngrok import ngrok
import threading

# FastAPI 실행을 별도 스레드에서 실행
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_api)
thread.start()

# ngrok을 통해 외부에서 접근 가능하도록 설정
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")


In [ ]:
import requests

# url = "https://{HOSTNAME}.ngrok-free.app/generate/"  # ngrok에서 출력된 URL 사용
url = "https://bca6aa5efa0a.ngrok-free.app/generate/"  # ngrok에서 출력된 URL 사용
data = {"prompt": "A futuristic cyberpunk city", "num_inference_steps": 50, "guidance_scale": 7.5}

response = requests.post(url, json=data)
image_data = response.json()["image"]

# Base64 디코딩하여 이미지 표시
from PIL import Image
import base64
from io import BytesIO

image = Image.open(BytesIO(base64.b64decode(image_data)))
type(image)
image.show()


In [ ]:
import requests
import base64
from PIL import Image
from io import BytesIO
from IPython.display import display  # ✅ Colab에서 이미지 표시

# ✅ 정확한 ngrok URL 입력
url = "https://bca6aa5efa0a.ngrok-free.app/generate/"

# ✅ 요청 데이터 JSON 형식 확인
data = {
    "prompt": "A futuristic cyberpunk city",
    "num_inference_steps": 50,
    "guidance_scale": 7.5
}

# ✅ 요청 보내기
response = requests.post(url, json=data, headers={'Content-Type': 'application/json'})

# ✅ 응답 확인 및 이미지 표시
if response.status_code == 200:
    try:
        image_data = response.json().get("image")

        if image_data:
            # ✅ Base64 디코딩하여 이미지 변환 및 Colab에서 표시
            image = Image.open(BytesIO(base64.b64decode(image_data)))
            display(image)  # ✅ Colab에서 즉시 이미지 표시
        else:
            print("❌ 오류: 'image' 키가 응답에 없습니다.")
            print(response.json())
    except Exception as e:
        print("❌ JSON 디코딩 오류:", str(e))
        print(response.text)
else:
    print(f"❌ 요청 실패! 상태 코드: {response.status_code}")
    print(response.text)
